In [0]:
begin
  declare qry_str string;

  if :load_type == 'full' then
    truncate table identifier(:sink_catalog || '.' || :sink_schema || '.' || :sink_table);
  end if;
  
  -- Build query string
  set qry_str = 
    "insert into " || :sink_catalog || '.' || :sink_schema || '.' || :sink_table || " "
    "select " || :select_list || " from " || :src_catalog || '.' || :src_schema || '.' || :src_table || " with(fetchSize = 100000) ";

  -- Add where clause
  if :load_type == 'incremental' then
    if :watermark_col_type = 'timestamp' then
      set qry_str = concat(qry_str, " where " || :watermark_col_name || " > '" || :watermark_col_start_value || "' <= '" || :watermark_col_end_value || "'");
    else
      set qry_str = concat(qry_str, " where " || :watermark_col_name || " > " || :watermark_col_start_value || " <= " || :watermark_col_end_value);
    end if;
  end if;
  
  execute immediate qry_str;
  select qry_str;
end